In [1]:
import sys
import cv2
import mediapipe as mp
import glob
import numpy as np
import json
import os
import time
from os.path import exists
from IPython.display import clear_output

In [10]:
# Data paths
json_file = '/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/WLASL_v0.3.json'
# txt_file = 'files10.txt'
# files_to_process = []

with open(json_file) as ipf:
    content = json.load(ipf)

# file1 = open(txt_file, 'r')
# temp = file1.readlines()
# files_to_process = [t[:-1] for t in temp]


# datapath = '/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/videos/'
datapath = '/mnt/HDD04/WASL Dataset/WLASL2000/'
vid_savepath = '/mnt/HDD04/WASL Dataset/Scripts/pose_annotated_videos/'
annot_path = '/mnt/HDD04/WASL Dataset/Scripts/pose_annotated_images/'
skelpath = '/mnt/HDD04/WASL Dataset/Scripts/pose_skeletons/'
frame_path = skelpath.replace('pose_skeletons', 'images')
json_path = skelpath.replace('pose_skeletons', 'jsons')

In [11]:
files = glob.glob(datapath + '*mp4')
print('Num files present:', len(files))
print('Num files should be:', sum([len(c['instances']) for c in content]))
print(files[0])

Num files present: 21095
Num files should be: 21083
/mnt/HDD04/WASL Dataset/WLASL2000/23865.mp4


In [12]:
print('Class:', content[0]['gloss'])
print('sample:', content[0]['instances'][0])

Class: book
sample: {'bbox': [385, 37, 885, 720], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 0, 'signer_id': 118, 'source': 'aslbrick', 'split': 'train', 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4', 'variation_id': 0, 'video_id': '69241'}


In [13]:
content[0]['instances'][0]['video_id']

'69241'

In [14]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [ ]:
# extract skeleton
start_time = time.time()
cnt = 0
for i, c in enumerate(content):
    for j in range(len(c['instances'])):
#         if c['instances'][j]['video_id'] not in files_to_process:
#             continue
        gloss = c['gloss']  # i.e., class 
        fname = datapath + c['instances'][j]['video_id'] + '.mp4'
        
        if exists(fname):
            cnt += 1
            now = time.time()
            clear_output(wait=True)
            print('Processing content ' + str(i+1) + '/' + str(len(content)) +
                  ', instance ' + str(j+1) + '/' + str(len(c['instances'])))
            cap = cv2.VideoCapture(fname)
            fps = cap.get(cv2.CAP_PROP_FPS)
#             start_frame = c['instances'][j]['frame_start'] - 1
            start_frame = 0
            end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
#             if c['instances'][j]['frame_end'] != -1:
#                 end_frame = c['instances'][j]['frame_end']
            subfolder_frame = frame_path + fname.split('/')[-1][:-4]
            if not exists(subfolder_frame):
                os.mkdir(subfolder_frame)
                bbox = c['instances'][j]['bbox']
                # extract frames
    #             for frame_no in range(start_frame, end_frame + 1):
                for frame_no in range(start_frame, end_frame + 1):
                    cap.set(1,frame_no)  # Where frame_no is the frame you want
                    ret, frame = cap.read()  # Read the frame
#                     frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                    cv2.imwrite(subfolder_frame + '/' + fname.split('/')[-1][:-4] + '_frame' + f'{frame_no:05d}' + '.jpg', frame)
            
            # MediaPipe
            subfolder_skel = skelpath + fname.split('/')[-1][:-4]
            if not exists(subfolder_skel):
                os.mkdir(subfolder_skel)
                IMAGE_FILES = sorted(glob.glob(subfolder_frame + '/*jpg'))
                with mp_pose.Pose(
                    static_image_mode=True,
                    model_complexity=2,
                    enable_segmentation=True,
                    min_detection_confidence=0.5) as pose:

                    image = cv2.flip(cv2.imread(IMAGE_FILES[0]), 1)
                    fname2 = IMAGE_FILES[0].split('/')[-1]
                    prev_results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    for idx, file in enumerate(IMAGE_FILES):
                        # Read an image, flip it around y-axis for correct handedness output (see
                        # above).
                        image = cv2.flip(cv2.imread(file), 1)
                        fname2 = file.split('/')[-1]
                        # Convert the BGR image to RGB before processing.
                        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                        # Print handedness and draw hand landmarks on the image.
    #                     print('Frame: ', idx, '/', len(IMAGE_FILES))
                        # print('Handedness:', results.multi_handedness)
                        if not results.pose_landmarks:
                            if not prev_results.pose_landmarks:
                                continue
                            else:
                                results = prev_results  # to prevent frame elimination due to non-existent hands
                        prev_results = results
                        image_height, image_width, _ = image.shape
                        annotated_image = image.copy()
                        data = []
#                         print(results.pose_landmarks.landmark)
#                         for pose_id, pose_landmarks in enumerate(results.pose_landmarks):


                          # print('hand_landmarks:', hand_landmarks)
                          # print(
                          #     f'Index finger tip coordinates: (',
                          #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                          #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                          # )
                          # print(dir(hand_landmarks.landmark[0]))
                          # print(hand_landmarks.landmark[0])
                          # print(hand_landmarks.landmark[0].x)

                        for land_id, mark in enumerate(results.pose_landmarks.landmark):

                            if land_id == 0:
                                x_temp = [mark.x]
                                y_temp = [mark.y]
                                z_temp = [mark.z]
                            else:
                                x_temp = np.concatenate((x_temp, [mark.x]), 0)
                                y_temp = np.concatenate((y_temp, [mark.y]), 0)
                                z_temp = np.concatenate((z_temp, [mark.z]), 0)

                        data.append(x_temp)
                        data.append(y_temp)
                        data.append(z_temp)
                        mp_drawing.draw_landmarks(
                              annotated_image,
                              results.pose_landmarks,
                              mp_pose.POSE_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                        
                        with open(subfolder_skel + '/' + fname2.replace('jpg', 'txt'), 'w') as f:
                            for elem in data:
                                for e in elem:
                                    f.write(str(e) + ' ')
                                f.write('\n')
                        subfolder_im = annot_path + fname.split('/')[-1][:-4]
                        if not exists(subfolder_im):
                            os.mkdir(subfolder_im)
                        cv2.imwrite(subfolder_im + '/' + fname2.replace('jpg', 'png'), cv2.flip(annotated_image, 1))
                    
                    # create video
#                     files = sorted(glob.glob(subfolder_im + '/*png'))
#                     frame = cv2.imread(files[0])
#                     height, width, layers = frame.shape
#                     video_name = vid_savepath + fname.split('/')[-1][:-4] + '.avi'
#                     video = cv2.VideoWriter(video_name, 0, fps, (width, height))
#                     for image in files:
#                         video.write(cv2.imread(image))
#                     video.release()
                    
            print('Elapsed time:', time.time() - now, 'sec')
            print('Num. processed file:' + str(cnt) + '/' + str(len(files)))
            print('Total Elapsed time:', (time.time() - start_time)/3600, 'hrs')
            # save content as json
            json_name = json_path + fname.split('/')[-1][:-3] + 'json'
            
            if not exists(json_name):
                content2 = c['instances'][j]
                content2['gloss'] = c['gloss']
                json_object = json.dumps(content2, indent=4)
                # Writing to sample.json
                with open(json_name, "w") as outfile:
                    outfile.write(json_object)

    

Processing content 5/2000, instance 21/26


In [8]:
with open(json_name) as ipf:
    temp_content = json.load(ipf)

In [9]:
temp_content

{'bbox': [210, 55, 567, 400],
 'fps': 25,
 'frame_end': -1,
 'frame_start': 1,
 'instance_id': 6,
 'signer_id': 12,
 'source': 'aslsearch',
 'split': 'train',
 'url': 'http://www.aslsearch.com/signs/videos/whistle.mp4',
 'variation_id': 0,
 'video_id': '63190',
 'gloss': 'whistle'}

In [10]:
from collections import Counter

[print(sum()) for c in content]
# Counter()

TypeError: sum() takes at least 1 positional argument (0 given)

In [ ]:
len(content[0]['instances'])